In the [previous instalment]({filename}20150330-Orientation_correlations_among_rice_grains-03.md) of this series, we obtained binned slices of the sample. [Fig.&nbsp;1](#fig01) below displays a typical example of these binned slices. We now want to segment the rice grains. However, the analysis (in particular, Otsu thresholding) might be perturbed by the fact that the walls of the sample container are visible on the 3D image. In this post, I will show how we can locate these walls. Then, any subsequent analysis will be performed within the Region Of Interest (ROI) thus defined.

## The circle Hough Transform

The sample container is cylindrical; since it was nearly vertical during the tomography experiment, its trace is a circular ring on each slice. We are going to use the [Circle Hough Transform](http://en.wikipedia.org/wiki/Circle_Hough_Transform) in order to locate the inner and outer circular boundaries which define this ring. To do so, we will use [python](https://www.python.org/), [numpy](http://www.numpy.org/) and [scikit-image](http://scikit-image.org/docs/dev/api/skimage.html). We first import these modules, and load the image

In [ ]:
import os.path

import numpy as np

from skimage.color import gray2rgb
from skimage.draw import circle_perimeter
from skimage.feature import canny
from skimage.io import imread, imsave
from skimage.transform import hough_circle
from skimage.util import img_as_ubyte

root = '.'
name = os.path.join(root, 'rice-bin_4x4x4-099.tif')
img = imread(name)

print('Read {}x{} image.'.format(*img.shape))

Then, we locate the edges, by means of a standard [Canny edge detector](http://en.wikipedia.org/wiki/Canny_edge_detector) (see also the [API docs](http://scikit-image.org/docs/dev/api/skimage.feature.html#canny) of scikit-image).

In [ ]:
edges = canny(img, sigma=0.)
imsave(os.path.join(root, 'rice-bin_4x4x4-edges-099.png'),
       img_as_ubyte(edges))

The resulting image is shown in [Fig.&nbsp;2](#fig02). It should be noted that due to the preliminary binning (which is nothing but a mean filter), the input image exhibits very little noise. Therefore, `sigma=0.0` in the above call to `skimage.feature.canny`.

We are now ready to compute the Circle Hough Transform. This transform aims at finding circles within an image. It was proposed by Duda and Hart ([1971](https://doi.org/10.1145/361237.361242)) (see also [Wikipedia](http://en.wikipedia.org/wiki/Hough_transform)). It should be understood as a histogram in parameter space. More precisely, a point $(x, y)$ belongs to the circle centered at $(c_x, c_y)$ and of radius $r$ if, and only if

$$(x-c_x)^2+(y-c_y)^2=r^2.$$

The circle under consideration is parameterized by $(c_x, c_y, r)$. Conversely, given a point $(x, y)$, the set of circles to which this point belongs is given by the triplet $(c_x, c_y, r)$ such that

$$(c_x-x)^2+(c_y-y)^2-r^2=0.$$

In the parameter space $(c_x, c_y, r)$, the set of circles to which point $(x, y)$ belongs is a *conical surface*. Its apex is $(x, y, 0)$, its axis is the $(0, 0, 1)$ direction and its aperture is 90°. As we are only interested in real circles, only the $r\geq0$ half-space should be considered.

How is this representation in parameter space to be used? We consider a binary (0/1) image; let $\{(x_i, y_i),i=1,\ldots,N\}$ denote the set of non-null pixels. We define $\mathcal H_i$ as the 3D, binary image of the cone associated in the sphere parameter space with pixel $(x_i, y_i)$. The Hough transform is then the (possibly normalized) sum of all $\mathcal H_i$

$$\mathcal H(c_x, c_y, r)=\sum_i\mathcal H_i(c_x, c_y, r).$$

$\mathcal H$ should really be understood as a *histogram*. Indeed, a peak in $\mathcal H$ indicates that the cones corresponding to many pixels intersect at the same point in parameter space. In other words, many pixels in the original image belong to the *same* circle. Finding circles in the original image therefore reduces to finding peaks in its Hough transform. That is what we are going to do presently. We must first compute the Hough transform. In order to reduce the CPU cost, we
will only consider these circles whose radius is close to that of the sample container. We saw in [part 2]({filename}20150310-Orientation_correlations_among_rice_grains-02.md) of this series that the radius of the sample container is 25 mm, while the voxel size is approximately 0.03 mm; after binning, the voxel size is therefore 0.12 mm, and the radius of the sample container is approx. 25 / 0.12 = 208 px. In the following code, we ask for the circle Hough transform for circles with radii between 190 and 220 px.

We first tell Python to ignore all warnings. This is extremely poor practice, but otherwise `skimage.io.imsave` will complain about low-contrast images (and my version of `skimage` does not implement the `check_contrast` keyword argument).

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
radii = np.arange(190, 220)
h = hough_circle(edges, radii)
for i, radius in enumerate(radii):
    imsave(os.path.join(root,
                        'rice-bin_4x4x4-hough-099-{}.tif'.format(radius)),
           h[i, ...])

The above code snippet saves a series of images, which is displayed below in 3D as an animated GIF (see [Fig.&nbsp;3](#fig03)). The two bright spots indicate the location of the inner and outer boundaries in the parameter space. We will use a very crude procedure to locate these two peaks (a more elaborate method is not required as we do not seek sub-pixel accuracy).

The code snippet below finds the two highest values of the Hough transform. The correct peak (inner boundary) corresponds to the *smallest* radius. Then, the coordinates of the center of the corresponding circle are found.

In [ ]:
h_max = np.max(h, axis=(1, 2))
k = np.min(np.argsort(h_max)[-2:])
radius = radii[k]
h = h[k, ...]
row, col = np.unravel_index(np.argmax(h), h.shape)
'The inner boundary is centered at ({} px, {} px); its radius is {} px.'.format(row, col, radius)

The radius of the inner boundary turns out to be *exactly* 208 pixels! For visual inspection, this circle is overlaid on top of the original image (adapted from this [scikit-image example](http://scikit-image.org/docs/dev/auto_examples/plot_circular_elliptical_hough_transform.html#example-plot-circular-elliptical-hough-transform-py)).

In [ ]:
img_rgb = gray2rgb(img_as_ubyte(img))
rows, cols = circle_perimeter(row, col, radius, method='andres')
img_rgb[rows, cols] = (255, 0, 0)
imsave(os.path.join(root, 'rice-bin_4x4x4-boundary-099.png'), img_rgb)

Which produces the following image (see [Fig.&nbsp;4](#fig04)).

A closer look (see [Fig.&nbsp;5](#fig05)) shows that we roughly achieved pixel accuracy, which will be sufficient for the analysis to come.

## Analysis of the whole stack

We are now ready to carry out the above analysis on all 172 images of the stack. This is what the script below does ([download]({static}20150529-Orientation_correlations_among_rice_grains-04/find_boundary.py)); in order to check that nothing went wrong, each image with overlaid boundary is saved for visual inspection.

This script saves the parameters of each circular boundary in an array, which can be restored for further analysis.

In [ ]:
a = np.load(os.path.join(root, 'circle_params.npy'))
print('avg = {}\nstd = {}'.format(a.mean(axis=0), a.std(axis=0)))

Which shows that there is very little variation of the circle parameters across the slices.

## Closing words

In this post, we saw how to define the (cylindrical) ROI on our stack of images. To do so, we used the Circle Hough Transform to find circular edges in the slices. In the [next]({filename}20150709-Orientation_correlations_among_rice_grains-05.md) instalment of this series, I will start discussing segmentation *per se*.

## Side-note: how to produce animated GIFs

The animated GIF in [Fig.&nbsp;3](#fig03) was produced with [ImageJ](http://imagej.nih.gov/ij/). The procedure is

1. Import the image sequence (`File → Import → Image Sequence…`) called
   `rice-bin_4x4x4-hough-099-*.tif`,
2. `Image → Stacks → 3D Project…`
3. `Image → Lookup Tables → Fire`
4. `File → Save As → Gif`

To change the frame rate, use `Image → Stacks → Tools → Animation Options…`